In [1]:
import os
import requests
import json
import openai

import yfinance as yf
from yahooquery import Ticker

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

/Users/altaf/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def get_stock_evolution(company_name, period="1y"):
    # Get the stock information
    stock = yf.Ticker(company_name)

    # Get historical market data
    hist = stock.history(period=period)

    # Convert the DataFrame to a string with a specific format
    data_string = hist.to_string()

    # Append the string to the "investment.txt" file
    with open("investment.txt", "a") as file:
        file.write(f"\nStock Evolution for {company_name}:\n")
        file.write(data_string)
        file.write("\n")

In [3]:
#get_stock_evolution("MSFT")

In [4]:
def get_financial_statements(ticker):
    # Create a Ticker object
    company = Ticker(ticker)

    # Get financial data
    balance_sheet = company.balance_sheet().to_string()
    cash_flow = company.cash_flow(trailing=False).to_string()
    income_statement = company.income_statement().to_string()
    valuation_measures = str(company.valuation_measures)  # This one might already be a dictionary or string

    # Write data to file
    with open("investment.txt", "a") as file:
        file.write("\nBalance Sheet\n")
        file.write(balance_sheet)
        file.write("\nCash Flow\n")
        file.write(cash_flow)
        file.write("\nIncome Statement\n")
        file.write(income_statement)
        file.write("\nValuation Measures\n")
        file.write(valuation_measures)

In [5]:
#get_financial_statements("MSFT")

In [6]:
def get_data(company_ticker, period="1y", filename="investment.txt"):
    # news = get_company_news(company_name)
    # if news:
    #     write_news_to_file(news, filename)
    # else:
    #     print("No news found.")

    get_financial_statements(company_ticker)

    #get_stock_evolution(company_ticker)

    

In [7]:
#get_data("MSFT")

In [19]:
def financial_analyst(request):

    filePath = "investment.txt"
    if os.path.exists(filePath):
        os.remove(filePath)

    print(f"Received request: {request}")
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=[{
            "role":
            "user",
            "content":
            f"Given the user request, what is the comapany name and the company stock ticker ?: {request}?"
        }],
        functions=[{
            "name": "get_data",
            "description":
            "Get financial data on a specific company for investment purposes",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type":
                        "string",
                        "description":
                        "The name of the company",
                    },
                    "company_ticker": {
                        "type":
                        "string",
                        "description":
                        "the ticker of the stock of the company"
                    },
                    "period": {
                        "type": "string",
                        "description": "The period of analysis"
                    },
                    "filename": {
                        "type": "string",
                        "description": "the filename to store data"
                    }
                },
                "required": ["company_name", "company_ticker"],
            },
        }],
        function_call={"name": "get_data"},
    )

    message = response["choices"][0]["message"]

    
    if message.get("function_call"):
        # Parse the arguments from a JSON string to a Python dictionary
        arguments = json.loads(message["function_call"]["arguments"])
        company_name = arguments["company_name"]
        company_ticker = arguments["company_ticker"]

        # Parse the return value from a JSON string to a Python dictionary
        get_data(company_ticker)

        with open("investment.txt", "r") as file:
            content = file.read()[:14000]

        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k",
            messages=[
                {
                    "role": "user",
                    "content": request
                },
                message,
                {
                    "role": "system",
                    "content": """write a detailled investment thesis to answer
                      the user request. Provide numbers to justify
                      your assertions, a lot ideally. Never mention
                      something like this:
                      However, it is essential to consider your own risk
                      tolerance, financial goals, and time horizon before
                      making any investment decisions. It is recommended
                      to consult with a financial advisor or do further
                      research to gain more insights into the company's f
                      undamentals and market trends. The user
                      already knows that"""
                },
                {
                    "role": "assistant",
                    "content": content,
                },
            ],
        )

        return second_response["choices"][0]["message"]["content"]

In [20]:
user_question = "How has been the financial performance of AAPL for the past 1 year been?"

print(financial_analyst(user_question))

Received request: How has been the financial performance of AAPL for the past 1 year been?
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_data",
    "arguments": "{\n\"company_name\": \"Apple Inc.\",\n\"company_ticker\": \"AAPL\",\n\"period\": \"1Y\"\n}"
  }
}
Over the past 1 year, the financial performance of Apple Inc. (AAPL) has been impressive. Let's examine some key financial metrics.

1. Stock Performance:
   - AAPL's stock price has shown strong growth over the past year, starting at around $135 and currently trading at around $125. The stock has experienced some fluctuations in the short term but has maintained an upward trend in the long run.

2. Revenue Growth:
   - Apple has continued to exhibit robust revenue growth. In its latest reported quarter, the company reported record-breaking revenue of $83.4 billion, representing a year-over-year increase of 29%. This growth can be attributed to higher sales across all product categories, includi

In [16]:
while True:
    user_question = input("Enter your investment request:\n\n")

    if user_question == 'exit':
        break

    print(financial_analyst(user_question))

Received request: What is the performance of AAPL?
As of the latest available data, the performance of Apple Inc. (AAPL) has been relatively positive. Here are some key points regarding AAPL's performance:

1. Stock Price Movement: AAPL's stock price has shown consistent growth over time. Since the provided dataset only goes up until December 2022, it is important to note that the stock price may have changed since then. However, historically, AAPL has shown strong performance and has been a popular choice among investors.

2. Upward Trend: AAPL has experienced an overall upward trend in its stock price, indicating positive market sentiment and investor confidence in the company. This can be seen by the increasing highs and lows in the stock price over time.

3. Volume: The trading volume for AAPL has been consistently high. This indicates strong investor interest and liquidity in the stock, as higher trading volumes suggest more market participation and potential buying/selling opport

Could not get exchangeTimezoneName for ticker '' reason: list index out of range
: No price data found, symbol may be delisted (period=1y)


AttributeError: 'str' object has no attribute 'to_string'